In [1]:
# Import libraries
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import os

In [10]:
# Set directory
path = "../Python_Scrape/" #กำหนด folder ที่จะให้ระบบทำงาน
os.chdir(path) #รัน Code เพื่อสั่งให้ระบบทำงานบน folder ที่ต้องการ

# URL of the webpage to scrape
url = 'https://th.indeed.com/jobs?q=data+analyst&l=Thailand&from=searchOnHP&vjk=f436453e1ab0b79f'

# Initialize a Chrome WebDriver instance 
driver = webdriver.Chrome(path+"chromedriver.exe")
driver.get(url)


C:\Users\akkarawin.sai\AppData\Local\Temp\ipykernel_23460\1516938698.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path+"chromedriver.exe")


In [11]:
# Get the HTML source code of the page after it has fully loaded
html = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find all job listings on the page
job_listings = soup.find_all('div', class_='job_seen_beacon')

In [12]:
# Initialize a list to store scraped records
records = []

In [13]:
def get_data(job_listing):
    # Extract job title
    title = job_listing.find("a").find("span").text.strip()
    
    # Extract company name if available, otherwise assign an empty string
    try:
        company = job_listing.find('span', class_='css-92r8pb eu4oa1w0').text.strip()
    except AttributeError:
        company = ''
    
    # Extract job location if available, otherwise assign an empty string
    try:
        location  = job_listing.find('div', class_='css-1p0sjhy eu4oa1w0').text.strip()
    except AttributeError:
        location = ''
        
    # Extract salary information if available, otherwise assign an empty string
    try:
        salary  = job_listing.find('div', class_='css-1cvvo1b eu4oa1w0').text.strip()
    except AttributeError:
        salary = ''
    
    # Extract job type if available, otherwise assign an empty string
    try:
        job_type = job_listing.find('div', class_='js-match-insights-provider-tvvxwd ecydgvn1').text.strip()
    except AttributeError:
        job_type = ''
    
    # Extract date posted
    date_posted = job_listing.find('span', class_='css-qvloho eu4oa1w0').text.strip()
    
    # Extract job summary
    summary = job_listing.find('div', class_='css-9446fg eu4oa1w0').text.strip()
    
    # Return a tuple containing all the extracted information
    return (title, company, location, salary, job_type, date_posted, summary)

In [14]:
# Loop to scrape data from multiple pages until there are no more pages available
while True:
    try:
        # Extract the URL of the next page if available
        url = 'https://th.indeed.com/' + soup.find('a', {'aria-label':'Next Page'}).get('href')
    except AttributeError:
        # If there are no more pages available, break the loop
        break
    
    # Open the next page in the browser
    driver.get(url)
    
    # Get the HTML source code of the next page
    html = driver.page_source
    
    # Parse the HTML of the next page using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find all job listings on the next page
    job_listings = soup.find_all('div', class_='job_seen_beacon') 

    # Iterate through each job listing on the page
    for job_listing in job_listings:
        # Extract data from the current job listing
        record = get_data(job_listing)
        
        # Append the extracted data to the records list
        records.append(record)



In [22]:
# Convert list of records into a DataFrame
df = pd.DataFrame(records, columns=['Title', 'Company', 'Location', 'Salary', 'Job Type', 'Date Posted', 'Summary'])

# Save DataFrame to a CSV file
df.to_csv('indeed_job_data.csv', index=False)

print("Data saved to job_data.csv")

Data saved to job_data.csv


In [21]:
df

,Title,Company,Location,Salary,Job Type,Date Posted,Summary
0,Data Analyst (ใช้โปรแกรม Excel),บริษัท เอ็นเอ็มที จำกัด,"บางกะปิ, กรุงเทพมหานคร",,,PostedPosted 30+ days ago,เข้าใจเรื่องสถิติ มีประสบการ์ณในการสรุปผล เช่น...
1,Data Analyst,AR Group,"คลองสาน, กรุงเทพมหานคร",,,PostedPosted 30+ days ago,Research for better improvements in data analy...
2,"Associate Analyst, Account Management Launch G...",Mastercard,กรุงเทพมหานคร,,,PostedPosted 30+ days ago,Support internal data research and analysis in...
3,Data Engineer (Junior level),NodeFlair,Thailand,,,PostedToday,Implement data models to support analysts.\nIm...
4,Data Analyst (SPX Express),Shopee,กรุงเทพมหานคร,,,PostedPosted 30+ days ago,As business intelligence specialists and data ...
...,...,...,...,...,...,...,...
589,เจ้าหน้าที่ฝ่ายบัญชี,บริษัท ออร์คิดจ๊อบ ดอท คอม จำกัด,"เมืองสมุทรสาคร, สมุทรสาคร",,,PostedPosted 30+ days ago,รายละเอียดงาน รูปแบบงาน : งานประจำ จำนวนที่รับ...
590,Senior Java Programmer,บริษัท กรุงไทย คอมพิวเตอร์ เซอร์วิสเซส จำกัด,กรุงเทพมหานคร,,,PostedPosted 30+ days ago,"To perform program and system analysis , syste..."
591,เจ้าหน้าที่บริหารงานขายสาขา พื้นที่ จ.มหาสารคา...,บริษัท เงินติดล้อ จำกัด (มหาชน),"พยัคฆภูมิพิสัย, มหาสารคาม",,,PostedPosted 30+ days ago,สามารถใช้คอมพิวเตอร์โปรแกรม MS Word และ MS Exc...
592,เจ้าหน้าที่บริหารงานขายสาขา สาขาวาปีปทุม จ.มหา...,บริษัท เงินติดล้อ จำกัด (มหาชน),"วาปีปทุม, มหาสารคาม",,,PostedPosted 30+ days ago,สามารถใช้คอมพิวเตอร์โปรแกรม MS Word และ MS Exc...
